# 1. JSONデータセットの読み込み
### COVID-19 Open Research Dataset Challenge (CORD-19) 作業用ノートブック

このノートブックは、CORD-19データセットの分析を容易に始められるようにするための、 [COVID-19 Open Research Dataset Challenge (CORD-19)](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) に対する作業用ノートブックです。  

<img src="https://miro.medium.com/max/3648/1*596Ur1UdO-fzQsaiGPrNQg.png" width="900"/>

アトリビューション:
* このノートブックで使用されるデータセットのライセンスは、[downloaded dataset](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge/download)に含まれるメタデータcsvに記載されています。
* 2020-03-03のデータセットには以下が含まれています。
  * `comm_use_subset`: 商用利用のサブセット (PMCコンテンツを含む) -- 9000 論文(内3論文は空), 186Mb
  * `noncomm_use_subset`: 非商用利用のサブセット (PMCコンテンツを含む) -- 1973 論文(内1論文は空), 36Mb
  * `biorxiov_medrxiv`: bioRxiv/medRxiv サブセット (ピアレビューされていない準備稿) -- 803 論文, 13Mb
* DatabricksあるいはDatabricksコミュニティエディションを使用する際には、`/databricks-datasets/COVID/CORD-19`からデータセットのコピーを利用できます。
* このノートブックは[CC BY 3.0](https://creativecommons.org/licenses/by/3.0/us/)のライセンスの下で共有することができます。

## パスの設定

`/databricks-datasets/COVID/CORD-19/2020-03-13/`からCORD-19 (2020-03-13)データセットを利用できます。

In [0]:
# ユーザーごとに一意のパスになるようにユーザー名をパスに含めます
import re
from pyspark.sql.types import * 

# Username を取得
username_raw = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
# Username の英数字以外を除去し、全て小文字化。Username をファイルパスやデータベース名の一部で使用可能にするため。
username = re.sub('[^A-Za-z0-9]+', '', username_raw).lower()

print(username)

takaakiyayoidatabrickscom

In [0]:
# Pythonにおけるパスの設定
comm_use_subset_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/"
noncomm_use_subset_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/"
biorxiv_medrxiv_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv/"
json_schema_path = "/databricks-datasets/COVID/CORD-19/2020-03-13/json_schema.txt"

In [0]:
display(dbutils.fs.ls(comm_use_subset_path))

path,name,size
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/000b7d1517ceebb34e1e3e817695b6de03e2fa78.json,000b7d1517ceebb34e1e3e817695b6de03e2fa78.json,12352
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/00142f93c18b07350be89e96372d240372437ed9.json,00142f93c18b07350be89e96372d240372437ed9.json,381675
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/0022796bb2112abd2e6423ba2d57751db06049fb.json,0022796bb2112abd2e6423ba2d57751db06049fb.json,312150
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/00326efcca0852dc6e39dc6b7786267e1bc4f194.json,00326efcca0852dc6e39dc6b7786267e1bc4f194.json,263583
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/00352a58c8766861effed18a4b079d1683fec2ec.json,00352a58c8766861effed18a4b079d1683fec2ec.json,131773
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/0043d044273b8eb1585d3a66061e9b4e03edc062.json,0043d044273b8eb1585d3a66061e9b4e03edc062.json,59255
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/0049ba8861864506e1e8559e7815f4de8b03dbed.json,0049ba8861864506e1e8559e7815f4de8b03dbed.json,214593
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/00623bf2715e25d3acacb3f210d6888ed840e3cb.json,00623bf2715e25d3acacb3f210d6888ed840e3cb.json,95935
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/0072159e1ebecc889e9bcabb58bb45c47e18a403.json,0072159e1ebecc889e9bcabb58bb45c47e18a403.json,175743
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/comm_use_subset/comm_use_subset/007618ad76a3548195ab5d11c1e2459931c91cd1.json,007618ad76a3548195ab5d11c1e2459931c91cd1.json,276393


In [0]:
display(dbutils.fs.ls(noncomm_use_subset_path))

path,name,size
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/0036b28fddf7e93da0970303672934ea2f9944e7.json,0036b28fddf7e93da0970303672934ea2f9944e7.json,725889
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/005c43980edf3fcc2a4d12ee7ad630ddb651ce6e.json,005c43980edf3fcc2a4d12ee7ad630ddb651ce6e.json,181544
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/006be99e337c84b8758591a54f0362353b24dfde.json,006be99e337c84b8758591a54f0362353b24dfde.json,175610
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/00a00d0edc750db4a0c299dd1ec0c6871f5a4f24.json,00a00d0edc750db4a0c299dd1ec0c6871f5a4f24.json,13831
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/00e5a723d44eb9f2698c38b518eff85c00f9753b.json,00e5a723d44eb9f2698c38b518eff85c00f9753b.json,31836
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/01297dffaf94c1314ca46088f7b829b8383c2f73.json,01297dffaf94c1314ca46088f7b829b8383c2f73.json,57266
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/013d9fb8719d3d3d47738f9f0604f3b643c4df57.json,013d9fb8719d3d3d47738f9f0604f3b643c4df57.json,105090
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/014e31dce7e3f2b1a7020a5debfbf228182f8b5e.json,014e31dce7e3f2b1a7020a5debfbf228182f8b5e.json,60903
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/0167dddb0e2783a60841b8e6f2b4e4cb981904e2.json,0167dddb0e2783a60841b8e6f2b4e4cb981904e2.json,40834
dbfs:/databricks-datasets/COVID/CORD-19/2020-03-13/noncomm_use_subset/noncomm_use_subset/018b5b5f732e955d349e14a83481739502ae104c.json,018b5b5f732e955d349e14a83481739502ae104c.json,100592


In [0]:
# シェル環境変数におけるパスの設定
import os
os.environ['comm_use_subset_path']=''.join(comm_use_subset_path)
os.environ['noncomm_use_subset_path']=''.join(noncomm_use_subset_path)
os.environ['biorxiv_medrxiv_path']=''.join(biorxiv_medrxiv_path)
os.environ['json_schema_path']=''.join(json_schema_path)
os.environ['username']=''.join(username)

## JSONスキーマの確認

以下のセルで表示しているように、これらのデータセットのスキーマは `json_schema.txt` で定義されています。

In [0]:
%sh 
cat /dbfs$json_schema_path

# JSON schema of full text documents


{
 "paper_id": <str>, # 40-character sha1 of the PDF
 "metadata": {
 "title": <str>,
 "authors": [ # list of author dicts, in order
 {
 "first": <str>,
 "middle": <list of str>,
 "last": <str>,
 "suffix": <str>,
 "affiliation": <dict>,
 "email": <str>
 },
 ...
 ],
 "abstract": [ # list of paragraphs in the abstract
 {
 "text": <str>,
 "cite_spans": [ # list of character indices of inline citations
 # e.g. citation "[7]" occurs at positions 151-154 in "text"
 # linked to bibliography entry BIBREF3
 {
 "start": 151,
 "end": 154,
 "text": "[7]",
 "ref_id": "BIBREF3"
 },
 ...
 ],
 "ref_spans": <list of dicts similar to cite_spans>, # e.g. inline reference to "Table 1"
 "section": "Abstract"
 },
 ...
 ],
 "body_text": [ # list of paragraphs in full body
 # paragraph dicts look the same as above
 {
 "text": <str>,
 "cite_spans": [],
 "ref_spans": [],
 "eq_spans": [],
 "section": "Introduction"
 },
 ...
 {
 ...,
 "section": "Conclusion"
 }
 ],
 "bib_entries": {
 "BIBREF0": {
 "ref_id": <str>,
 "title": <str>,
 "authors": <list of dict> # same structure as earlier,
 # but without `affiliation` or `email`
 "year": <int>,
 "venue": <str>,
 "volume": <str>,
 "issn": <str>,
 "pages": <str>,
 "other_ids": {
 "DOI": [
 <str>
 ]
 }
 },
 "BIBREF1": {},
 ...
 "BIBREF25": {}
 },
 "ref_entries":
 "FIGREF0": {
 "text": <str>, # figure caption text
 "type": "figure"
 },
 ...
 "TABREF13": {
 "text": <str>, # table caption text
 "type": "table"
 }
 },
 "back_matter": <list of dict> # same structure as body_text
 }
}

## Parquetパス変数の設定

`/tmp/<ユーザー名>/COVID/CORD-19/2020-03-13/`にParquetフォーマットで保存します。

In [0]:
# PythonにおけるParquetパスの設定
comm_use_subset_pq_path = f"/tmp/{username}/COVID/CORD-19/2020-03-13/comm_use_subset.parquet"
noncomm_use_subset_pq_path = f"/tmp/{username}/COVID/CORD-19/2020-03-13/noncomm_use_subset.parquet"
biorxiv_medrxiv_pq_path = f"/tmp/{username}/COVID/CORD-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv.parquet"

# シェル環境変数におけるパスの設定
os.environ['comm_use_subset_pq_path']=''.join(comm_use_subset_pq_path)
os.environ['noncomm_use_subset_pq_path']=''.join(noncomm_use_subset_pq_path)
os.environ['biorxiv_medrxiv_pq_path']=''.join(biorxiv_medrxiv_pq_path)

In [0]:
# クリーンアップ
print(f"/tmp/{username}/COVID/")
dbutils.fs.rm(f"/tmp/{username}/COVID/", True)

/tmp/takaakiyayoidatabrickscom/COVID/
Out[8]: True

## comm_use_subset JSONファイルの読み込み

これらは適切に整形されたJSONファイルのなので、これらのファイルを読み込むために`spark.read.json`を使用できます。*multiline*オプションを指定する必要があることに注意してください。

In [0]:
# comm_use_subset
comm_use_subset = spark.read.option("multiLine", True)\
  .json(comm_use_subset_path)

comm_use_subset.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMCID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: st

In [0]:
# レコード数 (JSONドキュメントの本来の数)
comm_use_subset.count()

Out[10]: 8997

### ファイル数の検証

In [0]:
%sh 
ls /dbfs$comm_use_subset_path | wc -l

9000

### comm_use_subset JSONファイルの保存

ファイル読み込みに時間を要するので、クエリー性能を改善するためにParquetファイルで保存しましょう。

In [0]:
# パーティション数の取得
comm_use_subset.rdd.getNumPartitions()

Out[12]: 286

In [0]:
print(comm_use_subset_pq_path)

/tmp/takaakiyayoidatabrickscom/COVID/CORD-19/2020-03-13/comm_use_subset.parquet

In [0]:
# 4パーティションでParquetフォーマットで書き出します 
# 今回のクラスターは4ノードであることを想定しています 
comm_use_subset.repartition(4).write.format("parquet").mode("overwrite").save(comm_use_subset_pq_path)

In [0]:
%sh 
ls -lsgA /dbfs/tmp/$username/COVID/CORD-19/2020-03-13/comm_use_subset.parquet

total 264307
 0 -rwxrwxrwx 1 root 0 May 27 00:42 _SUCCESS
 1 -rwxrwxrwx 1 root 420 May 27 00:42 _committed_439870986114650243
 0 -rwxrwxrwx 1 root 0 May 27 00:42 _started_439870986114650243
68354 -rwxrwxrwx 1 root 69994264 May 27 00:42 part-00000-tid-439870986114650243-9d76988d-17aa-45d3-a95c-ef793f1536e8-785-1-c000.snappy.parquet
66798 -rwxrwxrwx 1 root 68401081 May 27 00:42 part-00001-tid-439870986114650243-9d76988d-17aa-45d3-a95c-ef793f1536e8-787-1-c000.snappy.parquet
62761 -rwxrwxrwx 1 root 64266998 May 27 00:42 part-00002-tid-439870986114650243-9d76988d-17aa-45d3-a95c-ef793f1536e8-788-1-c000.snappy.parquet
66393 -rwxrwxrwx 1 root 67986196 May 27 00:42 part-00003-tid-439870986114650243-9d76988d-17aa-45d3-a95c-ef793f1536e8-786-1-c000.snappy.parquet

In [0]:
# 再度ファイルを読み込みます
comm_use_subset = spark.read.format("parquet").load(comm_use_subset_pq_path)

In [0]:
# レコード数 (JSONドキュメントの本来の数)
comm_use_subset.count()

Out[17]: 8997

## noncomm_use_subset JSONファイルの読み込み

これらは適切に整形されたJSONファイルのなので、これらのファイルを読み込むために`spark.read.json`を使用できます。*multiline*オプションを指定する必要があることに注意してください。

In [0]:
# noncomm_use_subset
noncomm_use_subset = spark.read.option("multiLine", True).json(noncomm_use_subset_path)
noncomm_use_subset.printSchema()

root
-- abstract: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- back_matter: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- cite_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- ref_spans: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- end: long (nullable = true)
 | | | |-- ref_id: string (nullable = true)
 | | | |-- start: long (nullable = true)
 | | | |-- text: string (nullable = true)
 | |-- section: string (nullable = true)
 | |-- text: string (nullable = true)
-- bib_entries: struct (nullable = true)
 |-- BIBREF0: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF1: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BIBREF10: struct (nullable = true)
 | |-- authors: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- first: string (nullable = true)
 | | | |-- last: string (nullable = true)
 | | | |-- middle: array (nullable = true)
 | | | | |-- element: string (containsNull = true)
 | | | |-- suffix: string (nullable = true)
 | |-- issn: string (nullable = true)
 | |-- other_ids: struct (nullable = true)
 | | |-- DOI: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- PMID: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | | |-- arXiv: array (nullable = true)
 | | | |-- element: string (containsNull = true)
 | |-- pages: string (nullable = true)
 | |-- ref_id: string (nullable = true)
 | |-- title: string (nullable = true)
 | |-- venue: string (nullable = true)
 | |-- volume: string (nullable = true)
 | |-- year: long (nullable = true)
 |-- BI

In [0]:
# レコード数 (JSONドキュメントの本来の数)
noncomm_use_subset.count()

Out[19]: 1972

In [0]:
%sh 
ls /dbfs$noncomm_use_subset_path | wc -l

1973

### noncomm_use_subset JSONファイルの保存

ファイル読み込みに時間を要するので、クエリー性能を改善するためにParquetファイルで保存しましょう。

In [0]:
# パーティション数の取得
noncomm_use_subset.rdd.getNumPartitions()

Out[21]: 63

In [0]:
# 4パーティションでPqarquetフォーマットで書き出します 
# ここではクラスターは4ノードであることを想定しています  
noncomm_use_subset.repartition(4).write.format("parquet").mode("overwrite").save(noncomm_use_subset_pq_path)

In [0]:
%sh
ls -lsgA /dbfs$noncomm_use_subset_pq_path

total 63857
 0 -rwxrwxrwx 1 root 0 May 27 00:46 _SUCCESS
 1 -rwxrwxrwx 1 root 424 May 27 00:46 _committed_7142725880382790491
 0 -rwxrwxrwx 1 root 0 May 27 00:46 _started_7142725880382790491
18347 -rwxrwxrwx 1 root 18787206 May 27 00:46 part-00000-tid-7142725880382790491-48de481d-3c03-4d91-8059-1643a3f0ba30-924-1-c000.snappy.parquet
15864 -rwxrwxrwx 1 root 16243724 May 27 00:46 part-00001-tid-7142725880382790491-48de481d-3c03-4d91-8059-1643a3f0ba30-926-1-c000.snappy.parquet
13628 -rwxrwxrwx 1 root 13954799 May 27 00:46 part-00002-tid-7142725880382790491-48de481d-3c03-4d91-8059-1643a3f0ba30-927-1-c000.snappy.parquet
16018 -rwxrwxrwx 1 root 16402115 May 27 00:46 part-00003-tid-7142725880382790491-48de481d-3c03-4d91-8059-1643a3f0ba30-925-1-c000.snappy.parquet

In [0]:
# ファイルの再読み込み
noncomm_use_subset = spark.read.format("parquet").load(noncomm_use_subset_pq_path)

In [0]:
# レコード数 (JSONドキュメントの本来の数)
noncomm_use_subset.count()

Out[25]: 1972

## biorxiv_medrxiv JSONファイルの読み込み

これらは適切に整形されたJSONファイルのなので、これらのファイルを読み込むために`spark.read.json`を使用できます。*multiline*オプションを指定する必要があることに注意してください。

In [0]:
# biorxiv_medrxiv
biorxiv_medrxiv = spark.read.option("multiLine", True).json(biorxiv_medrxiv_path)
biorxiv_medrxiv.count()

Out[26]: 803

In [0]:
# 4パーティションでPqarquetフォーマットで書き出します 
# ここではクラスターは4ノードであることを想定しています   
biorxiv_medrxiv.repartition(4).write.format("parquet").mode("overwrite").save(biorxiv_medrxiv_pq_path)

In [0]:
# ファイルの再読み込み
biorxiv_medrxiv = spark.read.format("parquet").load(biorxiv_medrxiv_pq_path)
biorxiv_medrxiv.count()

Out[28]: 803

クエリー性能改善のため、以降ではオリジナルのJSONファイルではなくParquetファイルを読み込みます。

# END